# Asignaciones financieras


## Datos de proyecto en análisis


In [240]:
proyecto = "sibayo"
mes = 5
anio = 2025

## Librerias necesarias


In [241]:
import pandas as pd
import os
import pprint
import pickle
import json
import xlsxwriter
from xlsxwriter.utility import xl_range, xl_rowcol_to_cell
import re
from typing import Dict, List, Any
import itertools
from pathlib import Path
from dotenv import load_dotenv, find_dotenv
from google.oauth2 import service_account
from google.cloud import firestore
import excel2img
from dotenv import load_dotenv, find_dotenv
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter
import win32com.client as win32

## Funciones utiles


### Calculadora de costo total


In [242]:
from typing import TypedDict


class CostoIngenieriaResult(TypedDict):
    costo_directo: float
    gastos_generales: float
    utilidad: float
    subtotal: float
    igv: float
    total: float


def calculadora_costo_total(costo_directo: float, gastos_generales: float) -> CostoIngenieriaResult:
    """
    Calcula el costo total de un proyecto de ingeniería civil en soles peruanos.

    Args:
        costo_directo (float): Costo directo del proyecto

    Returns:
        CostoIngenieriaResult: Diccionario con todos los valores calculados
    """
    # Calcular gastos generales (10% del costo directo)
    gastos_generales = round(costo_directo * gastos_generales, 2)

    # Calcular utilidad (5% del costo directo)
    utilidad = round(costo_directo * 0.05, 2)

    # Calcular subtotal
    subtotal = round(costo_directo + gastos_generales + utilidad, 2)

    # Calcular IGV (18% del subtotal)
    igv = round(subtotal * 0.18, 2)

    # Calcular total
    total = round(subtotal + igv, 2)

    return {
        "costo_directo": round(costo_directo, 2),
        "gastos_generales": gastos_generales,
        "utilidad": utilidad,
        "subtotal": subtotal,
        "igv": igv,
        "total": total,
    }

### Multiplicar el precio unitario actualizado por la carga trabajo


In [243]:
def calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo
):
    """
    Multiplica cada carga de trabajo del contratista por su precio unitario
    correspondiente.

    Si alguna clave en cargas_trabajo tiene carga > 0 y no existe
    en dict_precios_unitarios_actualizados, levanta KeyError.
    Claves con carga = 0 se ignoran (no dan error), y claves en precios que noprecios_uninatrios_expediente
    están en cargas se tratan con carga = 0.

    Args:
        dict_precios_unitarios_actualizados (dict): código → precio unitario.
        cargas_trabajo (dict): código → carga de trabajo.

    Returns:
        dict: código → precio total (precio_unitario * carga).

    Raises:
        KeyError: si alguna clave con carga > 0 en cargas_trabajo
                  no existe en dict_precios_unitarios_actualizados.
    """
    # Detectar faltantes solo para cargas > 0
    faltantes = {
        codigo
        for codigo, carga in cargas_trabajo.items()
        if carga != 0 and codigo not in dict_precios_unitarios_actualizados
    }
    if faltantes:
        raise KeyError(f"Faltan precios unitarios para las claves: {faltantes}")

    # Construir resultado: para cada precio, multiplicar por carga (0 si no hay)
    resultado = {}
    for codigo, precio_unitario in dict_precios_unitarios_actualizados.items():
        carga = cargas_trabajo.get(codigo, 0)
        resultado[codigo] = precio_unitario * carga

    return resultado

### Fusionar diccionarios

In [244]:
def fusionar_diccionarios(diccionario_de_diccionarios):
    """
    Fusiona diccionarios separados en un diccionario unificado.
    Solo incluye claves donde ambos valores sean diferentes de cero.
    Usa dinámicamente las claves del diccionario de entrada.
    
    Args:
        diccionario_de_diccionarios (dict): Diccionario con estructura:
            {
                "nombre_campo1": {clave: valor, ...},
                "nombre_campo2": {clave: valor, ...}
            }
    
    Returns:
        dict: Diccionario fusionado con estructura {clave: {nombre_campo1: valor, nombre_campo2: valor}}
    """
    # Obtener las claves (nombres de los campos) del diccionario principal
    nombres_campos = list(diccionario_de_diccionarios.keys())
    
    if len(nombres_campos) != 2:
        raise ValueError("El diccionario debe contener exactamente 2 campos")
    
    campo1_nombre = nombres_campos[0]
    campo2_nombre = nombres_campos[1]
    
    campo1_datos = diccionario_de_diccionarios[campo1_nombre]
    campo2_datos = diccionario_de_diccionarios[campo2_nombre]
    
    fusionado = {}
    
    # Obtener todas las claves únicas de ambos diccionarios
    todas_las_claves = set(campo1_datos.keys()) | set(campo2_datos.keys())
    
    for clave in todas_las_claves:
        valor1 = campo1_datos.get(clave, 0)
        valor2 = campo2_datos.get(clave, 0)
        
        # Solo agregar si ambos valores son diferentes de cero
        if valor1 != 0 and valor2 != 0:
            fusionado[clave] = {
                campo1_nombre: valor1,
                campo2_nombre: valor2
            }
    
    return fusionado

### Formatear progresiva

In [245]:
def formatear_progresiva(distancia, decimales=0):
    """
    Convierte una distancia en metros a notación de progresiva.
    
    Parámetros:
    - distancia: int o float, la distancia en metros.
    - decimales: int, número de decimales a mostrar en la parte de los metros.
    
    Retorna:
    - str: progresiva en formato 'K+XXX' con los decimales indicados.
    """
    if not isinstance(distancia, (int, float)):
        raise ValueError("La distancia debe ser un número (int o float).")
    if not isinstance(decimales, int) or decimales < 0:
        raise ValueError("Los decimales deben ser un entero no negativo.")
    
    km = int(distancia) // 1000
    metros = distancia - (km * 1000)
    
    formato_metros = f"{metros:0.{decimales}f}".zfill(3 + (1 if decimales > 0 else 0) + decimales)
    return f"{km}+{formato_metros}"

### Calcular avance

In [246]:
def calcular_avance(programado, ejecutado):
    """
    Calcula el porcentaje de avance basado en lo programado y lo ejecutado.

    Si lo programado es 0 y lo ejecutado es mayor a 0, devuelve 'ejecución adelantada'.
    Si ambos son 0, devuelve 0.0.
    En cualquier otro caso, devuelve el porcentaje (0-1) como float redondeado a 2 decimales.
    """
    if programado == 0:
        if ejecutado > 0:
            return "Ejec. adelantada"
        else:
            return 0.0
    else:
        porcentaje = (ejecutado / programado) 
        return porcentaje

### Ordenar por centena

In [247]:
def ordenar_por_centena(data: Any) -> List[Dict[str, Any]]:
    """
    Toma un dict (o un JSON en formato str) cuyas claves acaban en número
    y devuelve una lista de dicts {'key':…, 'value':…} ordenada por ese número.
    Compatible con Firestore (to_dict()) y JSON.
    """
    # Si viene como cadena JSON, lo convertimos
    if isinstance(data, str):
        data = json.loads(data)
    
    # Aseguramos que sea dict
    if not isinstance(data, dict):
        raise ValueError("Se esperaba un diccionario o un string JSON que represente un diccionario.")

    pattern = re.compile(r'(\d+)$')

    def obtener_clave_numerica(item):
        clave = str(item[0])  # Convertimos clave a string en caso no lo sea
        match = pattern.search(clave)
        if match:
            return int(match.group(1))
        else:
            return float('inf')  # Opcional: claves sin número al final se van al final

    sorted_items = sorted(data.items(), key=obtener_clave_numerica)

    return [{"key": k, "value": v} for k, v in sorted_items]

### Clave grupo

In [248]:
def clave_grupo(item):
    return int(item['key'][2:]) // 100  # 101→1, 201→2, etc.

## Carga de datos


### Firebase


In [249]:
# 1. Busca el .env en el directorio actual o en cualquiera de los padres
dotenv_path = find_dotenv()
if not dotenv_path:
    raise FileNotFoundError(
        "No se encontró ningún archivo .env en este directorio ni en sus padres."
    )
load_dotenv(dotenv_path)

# 2. Define el root del proyecto como la carpeta que contiene el .env
project_root = Path(dotenv_path).parent

# 3. Obtén la ruta relativa de las credenciales desde la variable de entorno
rel_cred_path = os.getenv("FIRESTORE_CREDENTIALS")
if not rel_cred_path:
    raise RuntimeError("No existe la variable FIRESTORE_CREDENTIALS en el .env")

# 4. Construye la ruta absoluta al JSON
cred_path = Path(rel_cred_path)
if not cred_path.is_absolute():
    cred_path = (project_root / cred_path).resolve()

if not cred_path.exists():
    raise FileNotFoundError(f"No existe el archivo de credenciales en: {cred_path}")

# 5. Carga las credenciales y crea el cliente de Firestore
credentials = service_account.Credentials.from_service_account_file(str(cred_path))
client = firestore.Client(credentials=credentials, project=credentials.project_id)

# 6. Prueba que funcione
print("Colecciones disponibles:", [c.id for c in client.collections()])

Colecciones disponibles: ['rutinarios']


In [250]:
db = firestore.Client(credentials=credentials, project=credentials.project_id)

#### Documento del proyecto firebase

In [251]:
doc_proyecto_firebase = db.collection("rutinarios").document(proyecto).get().to_dict()

pprint.pprint(doc_proyecto_firebase)

{'contrato': {'contratista': {'razon_social': 'PLATERS MANAGEMENT S.A.C.',
                              'ruc': ''},
              'denominacion_tramo_convenio': 'EMP. AR-111 (NUEVO SIBAYO) - '
                                             'TUTI EMP. AR-681 (DV. CHIVAY) '
                                             '(KM 32+252)',
              'fecha_inicio': {'anio': 2025, 'dia': 15, 'mes': 4},
              'id_contrato': '004-2025',
              'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                                     'dni': 0,
                                     'nombre': 'Genaro',
                                     'titulo': 'Ingeniero'},
              'monto_contrato': 165130.4,
              'numero_cuadrillas': 1,
              'numero_trabajadores': 3,
              'tiempo_ejecucion_dias': 240,
              'tipo_servicio': 'mantenimiento rutinario'},
 'datos_generales': {'distritos': ['Chivay', 'Tuti', 'Sibayo'],
                     'provincia': 'Cayllom

#### Contrato


In [252]:
contrato = doc_proyecto_firebase["contrato"]
pprint.pprint(contrato)

{'contratista': {'razon_social': 'PLATERS MANAGEMENT S.A.C.', 'ruc': ''},
 'denominacion_tramo_convenio': 'EMP. AR-111 (NUEVO SIBAYO) - TUTI EMP. AR-681 '
                                '(DV. CHIVAY) (KM 32+252)',
 'fecha_inicio': {'anio': 2025, 'dia': 15, 'mes': 4},
 'id_contrato': '004-2025',
 'jefe_mantenimiento': {'apellido': 'Tinta Cáceres',
                        'dni': 0,
                        'nombre': 'Genaro',
                        'titulo': 'Ingeniero'},
 'monto_contrato': 165130.4,
 'numero_cuadrillas': 1,
 'numero_trabajadores': 3,
 'tiempo_ejecucion_dias': 240,
 'tipo_servicio': 'mantenimiento rutinario'}


In [253]:
monto_contrato = contrato["monto_contrato"]
print(monto_contrato)

165130.4


#### Expediente técnico firebase

In [254]:
expediente_tecnico_firebase = doc_proyecto_firebase["expediente"]

pprint.pprint(expediente_tecnico_firebase)

{'cargas_trabajo': {'MR101': 10.28,
                    'MR103': 53.33,
                    'MR104': 85.33,
                    'MR201': 32350,
                    'MR202': 34.67,
                    'MR301': 15999,
                    'MR401': 64.67,
                    'MR601': 258.02,
                    'PP': 700,
                    'PS': 1512.58},
 'codigo_ruta': 'AR-683',
 'coordenadas': {'fin': {'altitud': 3629,
                         'datum': 'WGS84',
                         'hemisferio': 'S',
                         'progresiva': 32252.0,
                         'x': 220383.97,
                         'y': 8270075.15,
                         'zona': None,
                         'zona_letra': None},
                 'inicio': {'altitud': 3820,
                            'datum': 'WGS84',
                            'hemisferio': 'S',
                            'progresiva': 0,
                            'x': 226316.6,
                            'y': 8281146.79,
  

In [255]:
gastos_generales_expediente=expediente_tecnico_firebase['gastos_generales']
print(gastos_generales_expediente)


0.1


#### Progresiva de inicio y fin del expediente

In [256]:
progresiva_inicio=expediente_tecnico_firebase["coordenadas"]["inicio"]["progresiva"]
progresiva_fin=expediente_tecnico_firebase["coordenadas"]["fin"]["progresiva"]

print(progresiva_inicio)
print(progresiva_fin)

0
32252.0


#### Valorización programada mensual


In [257]:
desembolsos =  db.collection("rutinarios").document(proyecto).collection('presupuestos').document('desembolsos').get().to_dict()
cronogramas_desembolsos = desembolsos['cronograma_desembolsos']
pprint.pprint(cronogramas_desembolsos)

{'10': {'gastos_operativos': 704.82,
        'igv': 1075.15,
        'mantenimiento_con_go': 7753.01,
        'mantenimiento_con_igv': 7048.19,
        'mantenimiento_sin_igv': 5973.04},
 '11': {'gastos_operativos': 534.7,
        'igv': 815.65,
        'mantenimiento_con_go': 5881.72,
        'mantenimiento_con_igv': 5347.01,
        'mantenimiento_sin_igv': 4531.37},
 '12': {'gastos_operativos': 262.18,
        'igv': 399.93,
        'mantenimiento_con_go': 2883.94,
        'mantenimiento_con_igv': 2621.76,
        'mantenimiento_sin_igv': 2221.83},
 '4': {'gastos_operativos': 222.55,
       'igv': 339.48,
       'mantenimiento_con_go': 2448.06,
       'mantenimiento_con_igv': 2225.51,
       'mantenimiento_sin_igv': 1886.02},
 '5': {'gastos_operativos': 2553.4,
       'igv': 3895.02,
       'mantenimiento_con_go': 28087.43,
       'mantenimiento_con_igv': 25534.03,
       'mantenimiento_sin_igv': 21639.01},
 '6': {'gastos_operativos': 2988.84,
       'igv': 4559.25,
       'mantenim

In [258]:
desembolso_current_month = cronogramas_desembolsos[str(mes)]
print(desembolso_current_month["mantenimiento_con_igv"])

25534.03


#### Cargas de trabajo mensual presentadas por el contratista


In [259]:
valorizaciones = db.collection("rutinarios").document(proyecto).collection('valorizaciones').document(str(mes)).get().to_dict()
cargas_trabajo_contratista = valorizaciones['cargas_trabajo_contratista_inicial']
print(cargas_trabajo_contratista)

{'PP': 112, 'PS': 242.5, 'MR201': 5146.67, 'MR202': 5.1, 'MR103': 13.33, 'MR101': 0.86, 'MR301': 1999.88}


#### Cargas de trabajo del expediente técnico

In [260]:
cargas_trabajo_expediente_tecnico=doc_proyecto_firebase['expediente']['cargas_trabajo']

pprint.pprint(cargas_trabajo_expediente_tecnico)

{'MR101': 10.28,
 'MR103': 53.33,
 'MR104': 85.33,
 'MR201': 32350,
 'MR202': 34.67,
 'MR301': 15999,
 'MR401': 64.67,
 'MR601': 258.02,
 'PP': 700,
 'PS': 1512.58}


#### Precios unitarios del expediente técnico

In [261]:
precios_unitarios_expediente_tecnico = doc_proyecto_firebase['expediente']['precios_unitarios']

pprint.pprint(precios_unitarios_expediente_tecnico)

{'MR101': 317.8,
 'MR103': 19.07,
 'MR104': 21.18,
 'MR201': 0.53,
 'MR202': 95.34,
 'MR301': 0.17,
 'MR401': 13.28,
 'MR601': 2.44,
 'PP': 71.79,
 'PS': 47.01}


### Pickle

#### Cargas de trabajo programadas

In [262]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cargas_trabajo.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    cargas_trabajo_programadas_anualmente = pickle.load(f)

cargas_trabajo_programadas_anualmente.tail(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
codigo,,,,,,,,,,
MR111,92.230488,202.907073,184.460976,202.907073,202.907073,184.460976,202.907073,184.460976,55.338293,1512.58
MR112,52.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,87.500000,35.000000,700.00
MR201,2205.681818,4411.363636,3676.136364,4411.363636,4411.363636,3676.136364,4411.363636,3676.136364,1470.454545,32350.00
MR202,3.059118,5.098529,4.078824,4.078824,4.078824,4.078824,4.078824,4.078824,2.039412,34.67
MR203,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR204,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR206,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
MR301,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,1999.875000,0.000000,15999.00


In [263]:
#eliminando el total
cargas_trabajo_programadas_anualmente = cargas_trabajo_programadas_anualmente.iloc[:-1]

In [264]:
cargas_trabajo_programadas_current_month = cargas_trabajo_programadas_anualmente[f"2025-{mes:02d}"].to_dict()
pprint.pprint(cargas_trabajo_programadas_current_month)

{'MR101': 1.7133333333333332,
 'MR102': 0.0,
 'MR103': 13.3325,
 'MR104': 0.0,
 'MR111': 202.9070731707317,
 'MR112': 87.5,
 'MR201': 4411.363636363636,
 'MR202': 5.098529411764706,
 'MR203': 0.0,
 'MR204': 0.0,
 'MR205': 0.0,
 'MR206': 0.0,
 'MR301': 1999.875,
 'MR401': 0.0,
 'MR501': 0.0,
 'MR601': 0.0,
 'MR701': 0.0,
 'MR702': 0.0}


In [265]:
cargas_trabajo_programadas_anualmente.index.to_list()

['MR101',
 'MR102',
 'MR103',
 'MR104',
 'MR111',
 'MR112',
 'MR201',
 'MR202',
 'MR203',
 'MR204',
 'MR205',
 'MR206',
 'MR301',
 'MR401',
 'MR501',
 'MR601',
 'MR701',
 'MR702']

#### Cronograma anual

In [266]:
# Concatenar la ruta completa al archivo .pkl
ruta_archivo = os.path.join("data",proyecto, f"{proyecto}_cronograma_anual.pkl")

# Leer el archivo pickle
with open(ruta_archivo, "rb") as f:
    df_cronograma_anual = pickle.load(f)

df_cronograma_anual.head(15)

,2025-04,2025-05,2025-06,2025-07,2025-08,2025-09,2025-10,2025-11,2025-12,TOTAL
MR101,1,2,1,2,2,1,1,1,1,12
MR102,0,0,0,0,0,0,0,0,0,0
MR103,0,1,1,0,0,1,0,1,0,4
MR104,0,0,1,1,1,1,1,1,0,6
MR111,5,11,10,11,11,10,11,10,3,82
MR112,3,5,5,5,5,5,5,5,2,40
MR201,3,6,5,6,6,5,6,5,2,44
MR202,3,5,4,4,4,4,4,4,2,34
MR203,0,0,0,0,0,0,0,0,0,0
MR204,0,0,0,0,0,0,0,0,0,0


### JSON

#### Actividades

In [267]:
ruta_actividades= os.path.join("data", "general_data", "actividades.json")
with open(ruta_actividades, 'r', encoding='utf-8') as archivo:
    actividades = json.load(archivo)
# Ahora 'datos' es un diccionario de Python
print(actividades)

[{'key': 'MR100', 'value': {'label': 'Conservación de calzada', 'value': [{'key': 'MR101', 'value': {'label': 'Limpieza de calzada', 'carga_trabajo': 0, 'unidad': 'Km'}}, {'key': 'MR102', 'value': {'label': 'Bacheo', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR103', 'value': {'label': 'Desquinche', 'carga_trabajo': 0, 'unidad': 'm3'}}, {'key': 'MR104', 'value': {'label': 'Remoción de derrumbes', 'carga_trabajo': 0, 'unidad': 'm3'}}]}}, {'key': 'MR200', 'value': {'label': 'Limpieza de obras de arte', 'value': [{'key': 'MR201', 'value': {'label': 'Limpieza de cunetas', 'carga_trabajo': 1440, 'unidad': 'm'}}, {'key': 'MR202', 'value': {'label': 'Limpieza de alcantarillas', 'carga_trabajo': 0, 'unidad': 'unidad'}}, {'key': 'MR203', 'value': {'label': 'Limpieza de badén', 'carga_trabajo': 0, 'unidad': 'm2'}}, {'key': 'MR204', 'value': {'label': 'Limpieza de zanjas de coronación', 'carga_trabajo': 0, 'unidad': 'm'}}, {'key': 'MR205', 'value': {'label': 'Limpieza de pontones', 'carga_tra

## Cálculos


### Cargas de trabajo y pu de expediente tecnico

In [268]:
datos_cargas_trabajo_expediente_tecnico=fusionar_diccionarios(
    {
        'precio_unitario':precios_unitarios_expediente_tecnico,
        'carga_trabajo':cargas_trabajo_expediente_tecnico
    }
)

pprint.pprint(datos_cargas_trabajo_expediente_tecnico)

{'MR101': {'carga_trabajo': 10.28, 'precio_unitario': 317.8},
 'MR103': {'carga_trabajo': 53.33, 'precio_unitario': 19.07},
 'MR104': {'carga_trabajo': 85.33, 'precio_unitario': 21.18},
 'MR201': {'carga_trabajo': 32350, 'precio_unitario': 0.53},
 'MR202': {'carga_trabajo': 34.67, 'precio_unitario': 95.34},
 'MR301': {'carga_trabajo': 15999, 'precio_unitario': 0.17},
 'MR401': {'carga_trabajo': 64.67, 'precio_unitario': 13.28},
 'MR601': {'carga_trabajo': 258.02, 'precio_unitario': 2.44},
 'PP': {'carga_trabajo': 700, 'precio_unitario': 71.79},
 'PS': {'carga_trabajo': 1512.58, 'precio_unitario': 47.01}}


In [269]:
# Crear DataFrame usando pd.DataFrame.from_dict() con orient='index'
df_cargas_trabajo_expediente = pd.DataFrame.from_dict(
    datos_cargas_trabajo_expediente_tecnico, orient="index"
)

# Resetear el índice para convertir las claves MR en una columna
df_cargas_trabajo_expediente = df_cargas_trabajo_expediente.reset_index().rename(
    columns={"index": "codigo_MR"}
)

In [270]:
# Agregar columna parcial
df_cargas_trabajo_expediente["parcial"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

In [271]:
df_cargas_trabajo_expediente.head(15)

,codigo_MR,precio_unitario,carga_trabajo,parcial
0,MR601,2.44,258.02,629.5688
1,MR301,0.17,15999.00,2719.8300
2,MR101,317.80,10.28,3266.9840
3,MR202,95.34,34.67,3305.4378
4,MR401,13.28,64.67,858.8176
5,MR104,21.18,85.33,1807.2894
6,MR201,0.53,32350.00,17145.5000
7,MR103,19.07,53.33,1017.0031
8,PS,47.01,1512.58,71106.3858
9,PP,71.79,700.00,50253.0000


In [272]:
# Calcular el total
costo_directo = df_cargas_trabajo_expediente["parcial"].sum()
print("costo_directo", costo_directo)

costo_directo 152109.8165


In [273]:
costo_total_expediente = calculadora_costo_total(costo_directo,gastos_generales_expediente)
print(costo_total_expediente["total"])

206413.02


In [274]:
# Agregar columna parcial
df_cargas_trabajo_expediente["precio_unitario_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario"]
    * (monto_contrato / costo_total_expediente["total"])
)

df_cargas_trabajo_expediente["parcial_actualizado"] = (
    df_cargas_trabajo_expediente["precio_unitario_actualizado"]
    * df_cargas_trabajo_expediente["carga_trabajo"]
)

# Calcular el total
costo_directo_actualizado = df_cargas_trabajo_expediente["parcial_actualizado"].sum()

print(f"El costo directo es: {costo_directo_actualizado}")

El costo directo es: 121687.84140928515


In [275]:
df_cargas_trabajo_expediente.tail(10)

,codigo_MR,precio_unitario,carga_trabajo,parcial,precio_unitario_actualizado,parcial_actualizado
0,MR601,2.44,258.02,629.5688,1.952000,503.654991
1,MR301,0.17,15999.00,2719.8300,0.136000,2175.863789
2,MR101,317.80,10.28,3266.9840,254.239975,2613.586947
3,MR202,95.34,34.67,3305.4378,76.271993,2644.349984
4,MR401,13.28,64.67,858.8176,10.623999,687.054013
5,MR104,21.18,85.33,1807.2894,16.943998,1445.831380
6,MR201,0.53,32350.00,17145.5000,0.424000,13716.398671
7,MR103,19.07,53.33,1017.0031,15.255999,813.602401
8,PS,47.01,1512.58,71106.3858,37.607996,56885.103128
9,PP,71.79,700.00,50253.0000,57.431994,40202.396105


In [276]:
pago_costo_total_contratista = calculadora_costo_total(costo_directo_actualizado,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

165130.39


In [277]:
dict_precios_unitarios_actualizados = dict(
    zip(
        df_cargas_trabajo_expediente["codigo_MR"],
        df_cargas_trabajo_expediente["precio_unitario_actualizado"],
    )
)

pprint.pprint(dict_precios_unitarios_actualizados)

{'MR101': 254.23997536589505,
 'MR103': 15.255998521798674,
 'MR104': 16.9439983582431,
 'MR201': 0.4239999589173203,
 'MR202': 76.27199260976852,
 'MR301': 0.1359999868225367,
 'MR401': 10.623998970607571,
 'MR601': 1.9519998108646441,
 'PP': 57.431994435234756,
 'PS': 37.60799635604382}


### Cálculo de pago de acuerdo a cargas de trabajo


In [278]:
pago_costo_directo_parciales_contratista = calcular_costos_unitarios(
    dict_precios_unitarios_actualizados, cargas_trabajo_contratista
)
pprint.pprint(pago_costo_directo_parciales_contratista)

{'MR101': 218.64637881466973,
 'MR103': 203.3624602955763,
 'MR104': 0.0,
 'MR201': 2182.187868561005,
 'MR202': 388.98716230981944,
 'MR301': 271.9836536466547,
 'MR401': 0.0,
 'MR601': 0.0,
 'PP': 6432.383376746293,
 'PS': 9119.939116340627}


##### Visualizacion en dataframe


In [279]:
df_pago_costo_directo_parciales_contratista = pd.DataFrame.from_dict(
    pago_costo_directo_parciales_contratista, orient="index"
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.reset_index().rename(
        columns={"index": "codigo_MR"}
    )
)

df_pago_costo_directo_parciales_contratista = (
    df_pago_costo_directo_parciales_contratista.rename(columns={0: "monto_pago"})
)

df_pago_costo_directo_parciales_contratista

,codigo_MR,monto_pago
0,MR601,0.000000
1,MR301,271.983654
2,MR101,218.646379
3,MR202,388.987162
4,MR401,0.000000
5,MR104,0.000000
6,MR201,2182.187869
7,MR103,203.362460
8,PS,9119.939116
9,PP,6432.383377


In [280]:
# Ordenar el DataFrame por la columna 'monto_pago'
# Por defecto, el orden es ascendente (de menor a mayor)
df_ordenado = df_pago_costo_directo_parciales_contratista.sort_values(
    by="monto_pago", ascending=False
)

print("\nDataFrame Ordenado por 'monto_pago' (ascendente):")
print(df_ordenado)


DataFrame Ordenado por 'monto_pago' (ascendente):
  codigo_MR   monto_pago
8        PS  9119.939116
9        PP  6432.383377
6     MR201  2182.187869
3     MR202   388.987162
1     MR301   271.983654
2     MR101   218.646379
7     MR103   203.362460
0     MR601     0.000000
5     MR104     0.000000
4     MR401     0.000000


In [281]:
# Lambda que suma todos los valores de un diccionario
sumar_valores = lambda d: sum(d.values())

In [282]:
pago_costo_directo_contratista = sumar_valores(pago_costo_directo_parciales_contratista)
print(pago_costo_directo_contratista)

18817.490016714644


In [283]:
pago_costo_total_contratista = calculadora_costo_total(pago_costo_directo_contratista,gastos_generales_expediente)
print(pago_costo_total_contratista["total"])

25535.33


##### Validación


In [284]:
diferencia_costos = abs(
    pago_costo_total_contratista["total"]
    - desembolso_current_month["mantenimiento_con_igv"]
)

pprint.pprint(
    {
        "diferencia_costos": diferencia_costos,
        "ejecutado": pago_costo_total_contratista["total"],
        "programado": desembolso_current_month["mantenimiento_con_igv"],
    }
)

if diferencia_costos > 1:
    raise ValueError(
        "La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado"
    )

{'diferencia_costos': 1.3000000000029104,
 'ejecutado': 25535.33,
 'programado': 25534.03}


ValueError: La valorizacion de las actividades presentadas por el contratista no es coherente con lo programado

### Ordenando cargas de trabajo

In [ ]:
cargas_trabajo_ordenadas=ordenar_por_centena(cargas_trabajo_programadas_current_month)
pprint.pprint(cargas_trabajo_ordenadas)

[{'key': 'MR101', 'value': 0.99375},
 {'key': 'MR102', 'value': 328.0038461538461},
 {'key': 'MR103', 'value': 5.0075},
 {'key': 'MR104', 'value': 10.17142857142857},
 {'key': 'MR201', 'value': 0.0},
 {'key': 'MR202', 'value': 0.0},
 {'key': 'MR203', 'value': 44.448},
 {'key': 'MR204', 'value': 0.0},
 {'key': 'MR205', 'value': 0.0},
 {'key': 'MR206', 'value': 33.91},
 {'key': 'MR301', 'value': 3903.83},
 {'key': 'MR401', 'value': 0.0},
 {'key': 'MR501', 'value': 0.0},
 {'key': 'MR601', 'value': 0.0},
 {'key': 'MR701', 'value': 3.4320000000000004},
 {'key': 'MR702', 'value': 0.0}]
